In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
__author__ = 'Author'
__email__ = 'Email'

# Detecting Contradiction at the Lexical Level
## Dataset


In [4]:
# dependency
# # built-in
# import os, math, random, string
import random
# # # public
# import torch
# import torch.nn.functional as F
# import numpy as np
import pandas as pd
# from tqdm import tqdm, trange
# import nltk
# # nltk.download('words')
# from nltk.corpus import words
# from transformers import AutoModelForCausalLM, AutoTokenizer
from config import Config

%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Init

In [3]:
config = Config()
for k,v in config.__dict__.items():
    print(f'{k}: {v}')

seed: 0
llm: meta-llama/Llama-3.2-3B
CURR_PATH: ./
RESOURCE_PATH: ./res
DATA_PATH: ./res/data
RESULTS_PATH: ./res/results
LLMS_PATH: ./res/llms
LLM_PATH: ./res/llms/meta-llama/Llama-3.2-3B


# Wiki

## Sentence Complete - Capital

In [5]:
# load tsv
raw_df = pd.read_csv('res/data/wiki/capital50.tsv', sep='\t')
ids = raw_df['wikidata_id'].tolist()
capitals = raw_df['capital'].tolist()
countries = raw_df['country'].tolist()
raw_df.head()

,wikidata_id,country,capital,source
0,Q233,Malta,Valletta,The capital of Malta is
1,Q262,Algeria,Algiers,The capital of Algeria is
2,Q889,Afghanistan,Kabul,The capital of Afghanistan is
3,Q33,Finland,Helsinki,The capital of Finland is
4,Q736,Ecuador,Quito,The capital of Ecuador is


In [15]:
random.seed(config.seed)
h_template = 'The capital of {} is {}.'
hs = []
for country, capital in zip(countries, capitals):
    capitals_ = [c for c in capitals if c != capital]
    capital_ = random.choice(capitals_)
    h = h_template.format(country, capital_)
    hs.append(h)

In [16]:
raw_df['hopythesis'] = hs
raw_df.head()

,wikidata_id,country,capital,source,hopythesis
0,Q233,Malta,Valletta,The capital of Malta is,The capital of Malta is Brasília.
1,Q262,Algeria,Algiers,The capital of Algeria is,The capital of Algeria is Dakar.
2,Q889,Afghanistan,Kabul,The capital of Afghanistan is,The capital of Afghanistan is Astana.
3,Q33,Finland,Helsinki,The capital of Finland is,The capital of Finland is Kabul.
4,Q736,Ecuador,Quito,The capital of Ecuador is,The capital of Ecuador is Libreville.


## Contradiction Detection - Capital

In [6]:
# load tsv
raw_df = pd.read_csv('res/data/wiki/capital50.tsv', sep='\t')
raw_df.head()

,wikidata_id,country,capital,source
0,Q233,Malta,Valletta,The capital of Malta is
1,Q262,Algeria,Algiers,The capital of Algeria is
2,Q889,Afghanistan,Kabul,The capital of Afghanistan is
3,Q33,Finland,Helsinki,The capital of Finland is
4,Q736,Ecuador,Quito,The capital of Ecuador is


In [13]:
raw_ids = raw_df['wikidata_id'].tolist()

In [14]:
random.seed(config.seed)

ids, cos, cas, ps, hs, ys = [], [], [], [], [], []
country_list = raw_df['country'].tolist()
capital_list = raw_df['capital'].tolist()
for i, co, ca in zip(raw_ids, country_list, capital_list):
    ca_ = random.choice([_ for _ in capital_list if _ != ca])
    p = f'The capital of {co} is {ca}.'
    h = p
    h_ = f'The capital of {co} is {ca_}.'
    ps += [p, p]
    hs += [h, h_]
    ys += [False, True]
    ids += [i, i]
    cos += [co, co]
    cas += [ca, ca_]

In [15]:
# create a df
con_df = pd.DataFrame()
con_df['wikidata_id'] = ids
con_df['country'] = cos
con_df['capital'] = cas
con_df['premise'] = ps
con_df['hypothesis'] = hs
con_df['label'] = ys

In [16]:
con_df.head()

,wikidata_id,country,capital,premise,hypothesis,label
0,Q233,Malta,Valletta,The capital of Malta is Valletta.,The capital of Malta is Valletta.,False
1,Q233,Malta,Brasília,The capital of Malta is Valletta.,The capital of Malta is Brasília.,True
2,Q262,Algeria,Algiers,The capital of Algeria is Algiers.,The capital of Algeria is Algiers.,False
3,Q262,Algeria,Dakar,The capital of Algeria is Algiers.,The capital of Algeria is Dakar.,True
4,Q889,Afghanistan,Kabul,The capital of Afghanistan is Kabul.,The capital of Afghanistan is Kabul.,False


In [17]:
# save as tsv
con_df.to_csv('res/data/wiki/capital50_con.tsv', sep='\t', index=False)